In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

df = pd.read_csv('new_data.csv')
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [3]:
df.columns 

Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud'],
      dtype='object')

In [5]:
df.shape

(6362620, 11)

In [9]:
df.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06
mean,2.433972e+02,1.798619e+05,8.338831e+05,8.551137e+05,1.100702e+06,1.224996e+06,1.290820e-03,2.514687e-06
std,1.423320e+02,6.038582e+05,2.888243e+06,2.924049e+06,3.399180e+06,3.674129e+06,3.590480e-02,1.585775e-03
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.560000e+02,1.338957e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.390000e+02,7.487194e+04,1.420800e+04,0.000000e+00,1.327057e+05,2.146614e+05,0.000000e+00,0.000000e+00
75%,3.350000e+02,2.087215e+05,1.073152e+05,1.442584e+05,9.430367e+05,1.111909e+06,0.000000e+00,0.000000e+00
max,7.430000e+02,9.244552e+07,5.958504e+07,4.958504e+07,3.560159e+08,3.561793e+08,1.000000e+00,1.000000e+00


In [11]:
df.isnull().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [13]:
df['isFraud'].value_counts()  #0 = 6.3MILLION (NO FRAUD)  1 = 8213 (FRAUD DETECTED)

isFraud
0    6354407
1       8213
Name: count, dtype: int64

In [17]:
from sklearn.preprocessing import LabelEncoder
df['type']= LabelEncoder().fit_transform(df['type'])

In [19]:
df = df.drop(['nameOrig','nameDest'], axis=1)
df.head()

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,3,9839.64,170136.0,160296.36,0.0,0.0,0,0
1,1,3,1864.28,21249.0,19384.72,0.0,0.0,0,0
2,1,4,181.00,181.0,0.00,0.0,0.0,1,0
3,1,1,181.00,181.0,0.00,21182.0,0.0,1,0
4,1,3,11668.14,41554.0,29885.86,0.0,0.0,0,0


In [21]:
df['sender_org']= df['oldbalanceOrg'] - df['newbalanceOrig']

df['receiver_dest']=df['oldbalanceDest'] -df['newbalanceDest']	

In [23]:
x= df.drop(['isFraud','isFlaggedFraud'],axis=1)
y= df['isFraud']

In [25]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.2,random_state= 42)

In [ ]:
# APPLY SMOTE

In [31]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
x_train_resampled, y_train_resampled = smote.fit_resample(x_train,y_train)

In [97]:
print(y_train_resampled.value_counts())

isFraud
0    4448056
1    4448056
Name: count, dtype: int64


In [99]:
# REDUCE 10% OF THE DATASET
import pandas as pd

#COMBINE X AND Y
resampled_data = pd.concat([x_train_resampled, y_train_resampled],axis = 1)

#SAMPLE 10% OF DATA
resampled_sample = resampled_data.sample(frac=0.1, random_state= 42) # FRAC=0.1 MEANS SELECT 10% OF FULL DATA RANDOMLY

#SPLIT DATA INTO X AND Y FEATUES AND TARGET 
x_train_small = resampled_sample.drop('isFraud', axis = 1)
y_train_small = resampled_sample['isFraud']

In [54]:
# REDUCE 10% OF THE DATASET
import pandas as pd

#COMBINE X AND Y 
test_data = pd.concat([x_test, y_test],axis = 1)

#SAMPLE 10% OF DATA
test_sample = test_data.sample(n=3000, random_state= 42) 

#SPLIT DATA INTO X AND Y FEATUES AND TARGET 
x_test_small = test_sample.drop('isFraud', axis = 1)
y_test_small = test_sample['isFraud']

In [44]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(n_estimators=10, class_weight='balanced',random_state=42)
model.fit(x_train_small, y_train_small)

RandomForestClassifier(class_weight='balanced', n_estimators=10,
                       random_state=42)

In [48]:
y_pred= model.predict(x_test)

In [50]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
cm= confusion_matrix(y_test,y_pred)
acc= accuracy_score(y_test,y_pred)
report= classification_report(y_test,y_pred)

In [52]:
print('confusion_matrix:\n',cm)
print('accuracy_score:\n',acc)               
print('classification_report:\n',report)

confusion_matrix:
 [[1269198    1706]
 [     23    1597]]
accuracy_score:
 0.9986412829934839
classification_report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00   1270904
           1       0.48      0.99      0.65      1620

    accuracy                           1.00   1272524
   macro avg       0.74      0.99      0.82   1272524
weighted avg       1.00      1.00      1.00   1272524



In [71]:

#GET PREDICTED PROBA FOR CALSS 1
y_proba = model.predict_proba(x_test)[:,1]
threshold = 0.6
y_pred_custom = (y_proba >= threshold).astype(int)

In [101]:
from xgboost import XGBClassifier
model = XGBClassifier(random_state=42, use_label_encoder= False, eval_metric='logloss') # logloss to evaluate performance
model.fit(x_train_small, y_train_small)
y_pred= model.predict(x_test)

C:\Users\91961\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [19:56:27] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [102]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
cm= confusion_matrix(y_test,y_pred)
acc= accuracy_score(y_test,y_pred)
report= classification_report(y_test,y_pred)

In [105]:
print('confusion_matrix:\n',cm)
print('accuracy_score:\n',acc)               
print('classification_report:\n',report)

confusion_matrix:
 [[1901523    4828]
 [     20    2415]]
accuracy_score:
 0.9974601657807632
classification_report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00   1906351
           1       0.33      0.99      0.50      2435

    accuracy                           1.00   1908786
   macro avg       0.67      0.99      0.75   1908786
weighted avg       1.00      1.00      1.00   1908786



In [113]:
from xgboost import XGBClassifier
model = XGBClassifier(random_state=42, use_label_encoder= False, eval_metric='logloss') # logloss to evaluate performance
model.fit(x_train_small, y_train_small)
y_pred= model.predict(x_test)

C:\Users\91961\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:03:51] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [131]:
y_proba = model.predict_proba(x_test)[:,1]
threshold = 0.7
y_pred_custom = (y_proba >= threshold).astype(int)

In [133]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
cm= confusion_matrix(y_test,y_pred_custom)
acc= accuracy_score(y_test,y_pred_custom)
report= classification_report(y_test,y_pred_custom)

In [135]:
print('confusion_matrix:\n',cm)
print('accuracy_score:\n',acc)               
print('classification_report:\n',report)

confusion_matrix:
 [[1902845    3506]
 [     24    2411]]
accuracy_score:
 0.9981506570144584
classification_report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00   1906351
           1       0.41      0.99      0.58      2435

    accuracy                           1.00   1908786
   macro avg       0.70      0.99      0.79   1908786
weighted avg       1.00      1.00      1.00   1908786

